In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer
import grain

/Users/jack/jaximus/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
hf_ds = load_dataset("allenai/c4", "realnewslike", num_proc=4, split="train")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [3]:
parent_ds = (
    grain.MapDataset.source(hf_ds)
    .map(lambda x: {"tokens": tokenizer.encode(x["text"], return_tensors="np")[0]})
)

ds = grain.experimental.ConcatThenSplitIterDataset(
    parent=parent_ds,
    length_struct={"tokens": 1024},
)

In [3]:
from datasets import load_dataset
from transformers import AutoTokenizer
import grain
from typing import List
import numpy as np
def get_hf_dataset(
        hf_name: List[str],
        tokenizer_name: str,
        max_length: int,
        num_proc: int = 4,
        split: str = "train",
):
    hf_ds = load_dataset(*hf_name, split=split, num_proc=num_proc)
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

    parent_ds = (
        grain.MapDataset.source(hf_ds)
        .map(lambda x: {"tokens": tokenizer.encode(x["text"], return_tensors="np")[0]})
        
    )

    ds = grain.experimental.ConcatThenSplitIterDataset(
        parent=parent_ds,
        length_struct={"tokens": max_length+1},
    )

    ds = ds.map(lambda x: (x['tokens'][:-1], x['tokens'][1:]))

    return ds

/Users/jack/projects/jaximus/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
ds = get_hf_dataset(
    hf_name=["allenai/c4", "realnewslike"],
    tokenizer_name="gpt2",
    max_length=1024,
    num_proc=4,
    split="train",
)

ds = ds.batch(2)

for x, y in ds:
    print(x[0, :10])

Token indices sequence length is longer than the specified maximum sequence length for this model (1303 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1458 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1258 > 1024). Running this sequence through the model will result in indexing errors


[ 3260   262 28009  3438   286   520    13  7979   361   558]
[ 356  460 2139 1865 1194 1593 4843  286  674 7534]
[  884 22080  2223   329  7520    11   543   318  1521   523]
[ 4639   329   262  9382 21758   287 15127   338 49895   430]
[   46 16807    11   257  6466  1628   286  8884   290   262]
[6782   30 1867  546  262 4970 3025 1767 3354  389]
[  284   867  4671   326   389 25635   655  2029   393   655]
[  550   645  1103  2863   286 30024   422  1029  1524    13]
[   11 13387 24938    13  1439  1598   526   198     1  1212]
[24323   326   257  3741   393   379  1551  7930    13  3134]
[   13   379   262 10807   338  6252    68  9724 46134    82]
[8663 1700   12  774  278 1511 1830   13  198 1870]
[  262   749 28175 20790   983   314   447   247   303  1683]
[ 983 8034 3943 5684  329  720 1314 1510   13  198]
[  416   262  2351 24656  3854  4935   290 12549  7318    11]
[ 3329  2250   635 20825   284   262  2811 15896  3013 10735]
[  257   881 11622  4609  5386    13   198  2504

KeyboardInterrupt: 